In [1]:
from typing import Tuple

import numpy as np
from numpy.typing import NDArray
from scipy.integrate import quad

from pyzeal_algorithms.pyzeal_estimators.quad_estimator import QuadratureEstimator
from pyzeal_algorithms.pyzeal_estimators.estimator_cache import EstimatorCache
from pyzeal_utils.root_context import RootContext
from pyzeal_utils.pyzeal_containers.rounding_container import RoundingContainer

In [2]:
def testFunc(x: NDArray[np.complex128], n: int) -> NDArray[np.complex128]:
    """
    Simple function used to test the Newton polynomial based
    root finding algorithm
    """
    return n * x**(n - 1) / x**n

n = 2

In [3]:
# integrate the test function along the vertical strip [1j, rj]
realResult = quad(lambda x: np.real(testFunc(x * 1j, n)), 1, 2)
imagResult = quad(lambda x: np.imag(testFunc(x * 1j, n)), 1, 2)
integral = 1j * (realResult[0] + 1j * imagResult[0])
realError, imagError = realResult[1], imagResult[1]
print(f"for n={n}: integral={integral} -- error={realError + 1j * imagError}")

for n=2: integral=(1.3862943611198908-0j) -- error=1.5390959186233243e-14j


In [4]:
def integrateHorizontal(
    reRan: Tuple[float, float], imRan: Tuple[complex, complex]
) -> complex:
    """
    Integrate the (global) test function along a horizontal stretch in the
    complex plane.
    """
    realResult = quad(
        lambda x: np.real(testFunc(x + 1j * imRan[0], n)),
        reRan[0],
        reRan[1]
    )
    imagResult = quad(
        lambda x: np.imag(testFunc(x + 1j * imRan[0], n)),
        reRan[0],
        reRan[1]
    )
    return realResult[0] + 1j * imagResult[0]

In [5]:
def integrateVertical(
    reRan: Tuple[float, float], imRan: Tuple[complex, complex]
) -> complex:
    """
    Integrate the (global) test function along a vertical stretch in the
    complex plane.
    """
    realResult = quad(
        lambda x: np.real(testFunc(reRan[0] + 1j * x, n)),
        imRan[0],
        imRan[1]
    )
    imagResult = quad(
        lambda x: np.imag(testFunc(reRan[0] + 1j * x, n)),
        imRan[0],
        imRan[1]
    )
    return 1j * (realResult[0] + 1j * imagResult[0])

In [6]:
print(integrateHorizontal((1, 2), (2, 3)))
print(integrateVertical((1, 2), (2, 3)))

(0.4700036292457356-0.6435011087932843j)
(0.6931471805599454+0.2837941092083278j)


In [7]:
# calculate root by integrating in a rectangle around 0
# NOTE: must use logarithmic derivative here!
# NOTE: must account for direction of rectangle edges!
reRan = (-1, 1)
imRan = (-1, 1)
phi = integrateHorizontal(reRan, (-1, 1))
print(phi)
phi -= integrateHorizontal(reRan, (1, -1))
print(phi)
phi -= integrateVertical((-1, 1), imRan)
print(phi)
phi += integrateVertical((1, -1), imRan)
print(f"theoretical={n} -- actual={phi / (2 * np.pi * 1j)}")

3.141592653589793j
6.283185307179586j
9.42477796076938j
theoretical=2 -- actual=(2+0j)


In [2]:
cache = EstimatorCache()
estimator = QuadratureEstimator(cache)

In [5]:
n = 12
cache.reset()
moment = estimator.calcMoment(
    order=0,
    reRan=(-1, 1),
    imRan=(-1, 1),
    context=RootContext(lambda x: x**n, lambda x: n*x**(n-1), RoundingContainer(precision=(3, 3)))
)
print("Success!" if abs(moment / (2 * np.pi) - n) < 1e-5 else "Something went wrong... :(")

Success!


In [22]:
for n in [1, 2, 3, 4]:
    for order in [0, 1, 2, 3]:
        cache.reset()
        moment = estimator.calcMoment(
            order=order,
            reRan=(-1, 1),
            imRan=(-1, 1),
            context=RootContext(lambda x: (x-0.5)**n, lambda x: n*(x-0.5)**(n-1), RoundingContainer(precision=(3, 3)))
        )
        print(f"n={n}, order={order}, moment={round(moment / (2 * np.pi), 5)}")
    print("-" * 28)

n=1, order=0, moment=1.0
n=1, order=1, moment=0.5
n=1, order=2, moment=0.25
n=1, order=3, moment=0.125
---------------------------
n=2, order=0, moment=2.0
n=2, order=1, moment=1.0
n=2, order=2, moment=0.5
n=2, order=3, moment=0.25
---------------------------
n=3, order=0, moment=3.0
n=3, order=1, moment=1.5
n=3, order=2, moment=0.75
n=3, order=3, moment=0.375
---------------------------
n=4, order=0, moment=4.0
n=4, order=1, moment=2.0
n=4, order=2, moment=1.0
n=4, order=3, moment=0.5
---------------------------


In [3]:
from scipy.integrate import romb
import numpy as np

In [4]:
for n in [1, 2, 3, 4]:
    integrals = []
    for k in [8, 10, 12]:
        a, b = 0, 1j
        x = np.linspace(a, b, 2**k + 1)
        y = x ** n
        integrals.append(romb(y, abs(b - a) / (2**k)))
    print(f"n={n}, integrals={integrals[0]}, {integrals[1]}, {integrals[2]}")

n=1, integrals=0.5j, 0.5j, 0.5j
n=2, integrals=(-0.3333333333333333+0j), (-0.3333333333333333+0j), (-0.3333333333333333+0j)
n=3, integrals=-0.25j, -0.25j, -0.25j
n=4, integrals=(0.2+0j), (0.2+0j), (0.2+0j)


In [ ]:
# TODO: cache calculated arrays of function values in EstimatorCache!
# TODO: adjust complex line if root on line found!

# TODO: use higher moments to calculate the coefficients of Newton's polynomial
# TODO: use some rootfinder to calculate roots of Newton's polynomial

# TODO: Feature request for domains other than rectangles (custom Plugins)